# CIS 5450 Homework 3: Spark SQL

## Due: Monday, October 24, 2022 by 11:59pm ET

#### **100 points**

Welcome to CIS 5450 Homework 3! In this homework you will gain a mastery of using Spark SQL. Over the next few days you will be using an AWS EMR cluster to use Spark to manipulate the  `basic_stats.json` and `defensive.json` datasets, as well as `superbowl.csv`. Use this [link to access](https://drive.google.com/drive/folders/1yViGPJFB4NppRVXOzIiWvqdhK2BiJbQL?usp=sharing) the materials, though you don't need to now!

The goal of the homework will be to create a dataset for a time series model.  Yes, we are training you for the portions of the class coming up in the not-so-distant future!!

## Notes 
Before we begin here are some important notes to keep in mind,


1.  This notebook is specifically designed to be run on AWS SageMaker (and not on Google Colab). Please follow the instructions on the [Getting Started with AWS Academy](https://docs.google.com/document/d/1n9kK-FYL5dAptpiub9RfJ3XAJlI5NsGfJlEkiQD8uXA/edit?usp=sharing) guide to set up SageMaker and eventually, EMR. 
2.   **IMPORTANT!** I said it twice, it's really important. In this homework, we will be using AWS resources. You are given a quota ($100) to use for the entirety of the homework. There is a small chance you will use all this money, however it is important that at the end of every session, you **shut down your EMR cluster**.
3.   **Be sure you use SageMaker for this Homework** since we must connect to the EMR cluster and local Jupyter will have issues doing that. Using a SageMaker Notebook with an EMR cluster has two important abnormalities:
    * The first line of any cell in which you will use the spark session must be `%%spark`. Notice that all cells below have this.
    * You will, unfortunately, not be able to stop a cell while it is running. If you wish to do so, you will need to restart your cluster. See the Setup EMR Document for reference.
4.   You are **required** to use Spark SQL queries to handle the data in the assignment. Mastering SQL is more beneficial than being able to use Spark commands (functions) as it will show up in more areas of programming and data science/analytics than just Spark. Use the following [function list](https://spark.apache.org/docs/latest/api/sql/index.html#) to see all the SQL functions avaliable in Spark.
5.   Throughout the homework you will be manipulating Spark dataframes (sdfs). We do not specify any ordering on the final output. You are welcome to order your final tables in whatever way you deem fit. We will conduct our own ordering when we grade.
6. Based on the challenges you've faced in the previous homework, we are including information on the expected schema of your results.  Apache Spark is very fiddly but we hope this will help.
7. There are portions of this homework that are _very_ hard. We urge you start early to come to office hours and get help if you get stuck. But don't worry, I can see the future, and you all got this.

With that said, let's dive in!




## Step 0: Set up EMR

Follow the [Getting Started with AWS Academy](https://docs.google.com/document/d/1n9kK-FYL5dAptpiub9RfJ3XAJlI5NsGfJlEkiQD8uXA/edit?usp=sharing) guide for setting up the cluster.

Move on to Step 0.1 once you have a cluster (and IDE) set up. 


### Step 0.1: The Superfluous Setup

Run the following two cells. Apart from installing the usual dependencies (such as PennGrader), we will also install [sparkmagic](https://github.com/jupyter-incubator/sparkmagic), which allows your (Jupyter) notebook to connect to and use your EMR Cluster through [Livy](https://livy.incubator.apache.org), a REST service for Spark.

In [1]:
!pip install penngrader

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [2]:
%load_ext sparkmagic.magics 

### Step 0.2: The Splendid Spark Connection 

Now, connect your notebook to the EMR cluster you created. In the first cell, copy the link to the Master Public DNS specified in the setup document.

For example, if my URL (directly from the AWS CloudFormation's "Outputs" tab) is `	http://ec2-44-201-74-248.compute-1.amazonaws.com` and my password was `password1`, I'd run: 

```
%spark add -s spark_session -l python -u http://ec2-44-201-74-248.compute-1.amazonaws.com -a cis545-livy -p password1 -t Basic_Access
```

**TODO**: Substitute the url and your password in the above example line in the cell below 


In [6]:
# TODO: Copy the line above, enter your Master Public DNS with the proper formatting and host, and update the password
%spark add -s spark_session -l python -u http://ec2-3-94-88-156.compute-1.amazonaws.com -a cis545-livy -p 123456 -t Basic_Access

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1666724100803_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


In [4]:
# If you ever need to restart, you may need to...
# %spark delete -s my_session
#OR just factory reset runtime under the runtime tab

**TODO**: Enter your 8-digit Penn Key as an integer in the cell 
below. This will be used in the autograder.  **Please also update the cell below, with the same ID!**

In [7]:
from penngrader.grader import *
STUDENT_ID = 10760059

In [8]:
grader = PennGrader(homework_id = 'CIS_5450O_Fall22_HW3', student_id = STUDENT_ID)

PennGrader initialized with Student ID: 10760059

Make sure this correct or we will not be able to store your grade


**Please make sure you also update this one, so the grader can similarly be updated on Spark/EMR!**

In [9]:
%%spark
from penngrader.grader import *
STUDENT_ID =  10760059
grader = PennGrader(homework_id = 'CIS_5450O_Fall22_HW3', student_id = STUDENT_ID)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

PennGrader initialized with Student ID: 10760059

Make sure this correct or we will not be able to store your grade

Run the above cells to setup the autograder in BOTH the Spark session as well as locally, make sure to have set your 8 digit Penn ID in the cell above. It will also import all the modules you need for the homework.

_Note_: Since we are using an EMR cluster we will only have access to some of modules that exist for Python, meaning things like `pandas`, `numpy`, etc. may not all be available. We have written the entire homework such that the solution does not require any of these.

## Step 1: Data Wrangling, Cleaning, and Shaping

It's football fever! We recently witnessed Super Bowl LVI, in which Los Angeles Rams trumped the Cincinnati Bengals to clinch the title at their brand new home stadium. To continue the hype, we thought it would be exciting for you to work on NFL data and that's what we will be doing today.

<br>

![Winners](https://imagez.tmz.com/image/d6/4by3/2022/02/14/d62fc8c540c348c0aa75730b9c9a5b8d_md.jpg)

<br>

The data you will use is stored in an AWS S3 bucket, a cloud storage service. You now need to download it onto the nodes of your [EMR cluster](https://docs.aws.amazon.com/emr/latest/ManagementGuide/emr-what-is-emr.html). 

### Step 1.1: The Stupendous Schema

When loading data, Spark will try to infer its structure on its own. This process is faulty because Spark will sometimes infer the type incorrectly. Spark's ability to determine types is not reliable, thus you will need to define a schema for `basic_stats2.json` and `defensive3.json`.

A schema is a description of the structure of data. We have given you an example with `defensive3.json` and you will be defining an explicit schema for `basic_stats2.json`. 


In Spark, schemas are defined using a `StructType` object. This is a collection of data types, termed `StructField`'s, that specify the structure and variable type of each component of the dataset. For example, suppose we have the following simple JSON object:
```json
{
 "student_name": "Data Wrangler",
 "GPA": 1.4,
 "courses": [
    {"department": "Computer and Information Science",
     "course_id": "CIS 545",
     "semester": "Fall 2021"},
    {"department": "Computer and Information Science",
     "course_id": "CIS 555",
     "semester": "Fall 2021"}
 ],
 "grad_year": 2022
}
```

We would define its schema as follows:

```python       
schema = StructType([
           StructField("student_name", StringType(), nullable=True),
           StructField("GPA", FloatType(), nullable=True),
           StructField("courses", ArrayType(
                StructType([
                  StructField("department", StringType(), nullable=True),
                  StructField("course_id", StringType(), nullable=True),
                  StructField("semester", StringType(), nullable=True)
                ])
           ), nullable=True),
           StructField("grad_year", IntegerType(), nullable=True)
         ])
```


Each `StructField` has the following structure: `(name, type, nullable)`. The `nullable` flag defines that the specified field may be empty. Your first task is to define the `schema` of `basic_stats2.json`. You can take a look at the schema for `defensive3.json` for reference. A smaller version of the JSON dataset can be found here. [defensive3.json](https://drive.google.com/file/d/1B75g3-GEdrXTiWn9069bDwvIia9xQIQR/view?usp=sharing), [basic_stats2.json](https://drive.google.com/file/d/1aF4fnH4JI_r8wWN27qFdyjiXWOs4mlg9/view?usp=sharing).




There is also no grading cell for this step.  But your JSON file won't load if it's wrong, so you have a way of testing.

**TODO**: Create a schema for ``basic_stats2.json``

In [10]:
%%spark

from pyspark.sql.types import *

#schema for defensive.json

schema_defensive = StructType([
                     
                     StructField("Player Id", StringType(), nullable = True),
                     StructField("Name", StringType(), nullable = True),
                     StructField("Position", StringType(), nullable = True),
                     StructField("Year", IntegerType(), nullable = True),
                     StructField("Team", StringType(), nullable = True),
                     StructField("Games Played", StringType(), nullable = True),
                     StructField("Total Tackles", StringType(), nullable = True),
                     StructField("Solo Tackles", StringType(), nullable = True),
                     StructField("Assisted Tackles", StringType(), nullable = True),
                     StructField("Sacks", StringType(), nullable = True),
                     StructField("Safties", StringType(), nullable = True),
                     StructField("Passes Defended", StringType(), nullable = True),
                     StructField("Ints", StringType(), nullable = True),
                     StructField("Ints for TDs", StringType(), nullable = True),
                     StructField("Int Yards", StringType(), nullable = True),
                     StructField("Yards Per Int", StringType(), nullable = True),
                     StructField("Longest Int Return", StringType(), nullable = True),            
                ])


#TODO: Create a schema for basic_stats2.json
         #TODO: Create a schema for basic_stats2.json
schema_basic_stats2 = StructType([
                     
                     StructField("Age", StringType(), nullable = True),
                     StructField("Birth Place", StringType(), nullable = True),
                     StructField("Birthday", StringType(), nullable = True),
                     StructField("College",StringType(), nullable = True),
                     StructField("Current Status", StringType(), nullable = True),
                     StructField("Current Team", StringType(), nullable = True),
                     StructField("Experience", StringType(), nullable = True),
                     StructField("Height (inches)", StringType(), nullable = True),
                     StructField("High School", StringType(), nullable = True),
                     StructField("High School Location", StringType(), nullable = True),
                     StructField("Name", StringType(), nullable = True),
                     StructField("Number", StringType(), nullable = True),
                     StructField("Player Id", StringType(), nullable = True),
                     StructField("Position", StringType(), nullable = True),
                     StructField("Weight (lbs)", StringType(), nullable = True),
                     StructField("Years Played", StringType(), nullable = True),        
                ])           

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Step 1.2: The Langorous Load

In the following cell, load the `basic_stats2.json` dataset and the `defensive3.json` from GCP's storage service into Spark dataframes (sdf) called `basic_stats_sdf` and `defensive_sdf` respectively. If you have constructed `schema1` and `schema2` correctly, `spark.read.json()` will read in the dataset. 

***You do not need to edit this cell***. If this doesn't work, go back to the prior cell and update your schema.

In [11]:
%%spark

defensive_stats_sdf = spark.read.json("s3://penn-cis545-files/defensive3.json", schema=schema_defensive, multiLine = True, primitivesAsString = True)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
%%spark
# Let's print out the first few rows to see how the data looks like in tabular form
defensive_stats_sdf.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------------+--------+----+-----------------+------------+-------------+------------+----------------+-----+-------+---------------+----+------------+---------+-------------+------------------+
|         Player Id|          Name|Position|Year|             Team|Games Played|Total Tackles|Solo Tackles|Assisted Tackles|Sacks|Safties|Passes Defended|Ints|Ints for TDs|Int Yards|Yards Per Int|Longest Int Return|
+------------------+--------------+--------+----+-----------------+------------+-------------+------------+----------------+-----+-------+---------------+----+------------+---------+-------------+------------------+
|quinnjohnson/79593|Johnson, Quinn|        |2013| Tennessee Titans|           4|            0|          --|              --|   --|     --|             --|  --|          --|       --|            0|                --|
|quinnjohnson/79593|Johnson, Quinn|        |2012| Tennessee Titans|          16|            1|           1|               0|    0|     -

**TODO**: Create a spark dataframe for ``basic_stats2``

In [13]:
%%spark

#TODO: Create a spark dataframe for basic_stats2
basic_stats_sdf = spark.read.json("s3://penn-cis545-files/basic_stats2.json", schema=schema_basic_stats2, multiLine = True, primitivesAsString = True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

The cell below shows how to run SQL commands on Spark tables. Use this as a template for all your SQL queries in this notebook. 

***You do not need to edit this cell***.

In [14]:
%%spark

import pandas as pd

# Create SQL-accesible table
basic_stats_sdf.createOrReplaceTempView("basic_stats")

# Declare SQL query to be excecuted
query = '''SELECT * 
           FROM basic_stats ORDER BY `Player Id` LIMIT 10'''

# Save the output sdf of spark.sql() as answer_sdf and convert to Pandas
answer_basic_sdf = spark.sql(query)
answer_basic_sdf.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+---------------+---------+--------------------+--------------+--------------------+----------+---------------+--------------------+--------------------+-----------------+------+--------------------+--------+------------+------------+
| Age|    Birth Place| Birthday|             College|Current Status|        Current Team|Experience|Height (inches)|         High School|High School Location|             Name|Number|           Player Id|Position|Weight (lbs)|Years Played|
+----+---------------+---------+--------------------+--------------+--------------------+----------+---------------+--------------------+--------------------+-----------------+------+--------------------+--------+------------+------------+
|  45|   Griffin , GA|10/8/1971|       Florida State|       Retired|                    | 2 Seasons|             73|                    |                    |   Ellison, 'Omar|  null|'omarellison/2500540|        |         200| 1995 - 1996|
|  22|Fort Worth , TX|3/21/1995|        

We will copy the `answer_sdf` to Colab to submit to PennGrader...

In [15]:
%%spark

## 4 points
## AUTOGRADER Step 1.2:   
grader.grade(test_case_id = 'first', answer = pd.read_json(answer_basic_sdf.toPandas().to_json()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Correct! You earned 4/4 points. You are a star!

Your submission has been successfully recorded in the gradebook.

**TODO**: Now, implement the exact same thing yourself for defensive_stats_sdf.

In [16]:
%%spark

# TODO: Create SQL-accesible table

defensive_stats_sdf.createOrReplaceTempView("defensive_stats")

#TODO: Declare SQL query to be excecuted

query = '''SELECT * 
           FROM defensive_stats ORDER BY `Player Id` LIMIT 10'''
#TODO: Save the output sdf of spark.sql() as answer_defensive_sdf and convert to Pandas

answer_defensive_sdf = spark.sql(query)
answer_defensive_sdf.show()



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------------+--------+----+--------------------+------------+-------------+------------+----------------+-----+-------+---------------+----+------------+---------+-------------+------------------+
|           Player Id|             Name|Position|Year|                Team|Games Played|Total Tackles|Solo Tackles|Assisted Tackles|Sacks|Safties|Passes Defended|Ints|Ints for TDs|Int Yards|Yards Per Int|Longest Int Return|
+--------------------+-----------------+--------+----+--------------------+------------+-------------+------------+----------------+-----+-------+---------------+----+------------+---------+-------------+------------------+
|a'shawnrobinson/2...|Robinson, A'Shawn|      DT|2016|       Detroit Lions|          16|           30|          22|               8|    2|     --|              7|  --|          --|       --|            0|                --|
|   a.j.bouye/2541162|      Bouye, A.J.|      CB|2016|      Houston Texans|          15|           63|  

In [17]:
%%spark

## 4 points

## AUTOGRADER Step 1.2:   
grader.grade(test_case_id = 'second', answer = pd.read_json(answer_defensive_sdf.toPandas().to_json()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Correct! You earned 4/4 points. You are a star!

Your submission has been successfully recorded in the gradebook.

**TODO**: In the next cell, create `step_1_2_sdf` to fetch the data from the above basic_stats table, returning rows with schema `(Player Id, Name)`, in **lexicographical order** of `Name`.  Limit your sdf to 10 rows. Save your final answer to Colab to submit to PennGrader, as demonstrated above.

In [18]:
%%spark 

# TODO: create step_1_2_sdf
query = '''SELECT `Player Id`, `Name`
           FROM basic_stats 
           ORDER BY `Name` LIMIT 10'''

step_1_2_sdf = spark.sql(query)
step_1_2_sdf.show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------------+
|           Player Id|             Name|
+--------------------+-----------------+
|isaakoaaitui/2531731|   Aaitui, Isaako|
|   jonabbate/2495524|      Abbate, Jon|
|    joeabbey/2508144|       Abbey, Joe|
|  fayeabbott/2508147|     Abbott, Faye|
| vinceabbott/2508148|    Abbott, Vince|
|jaredabbrederis/2...|Abbrederis, Jared|
|dukeabbruzzi/2508149|   Abbruzzi, Duke|
| naderabdallah/89680|  Abdallah, Nader|
|mehdiabdesmad/255...|  Abdesmad, Mehdi|
|isaabdul-quddus/2...|Abdul-Quddus, Isa|
+--------------------+-----------------+

In [19]:
%%spark

# 5 points
grader.grade(test_case_id = 'lex_10_ids_last_names', answer = step_1_2_sdf.toPandas())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Correct! You earned 5/5 points. You are a star!

Your submission has been successfully recorded in the gradebook.

### Step 1.3: Further Cleaning

**TODO**: Now, we can see that our data is a little ill-formatted, so let us do some cleaning for our two dataframes. Here is what you will do:

For Defensive:

1. Drop all the rows that have null values in Total Tackles and Games Played.
2. We only want to consider data post 2000 (inclusive).


Save this in `defensive_cleaned_sdf`.

Once you have done that, we want to select the basic stats only for those filtered players in `defensive_cleaned_sdf` and create a cumulated dataframe called `defensive_player_stats_sdf`.

For defensive_player_stats, you need attributes from both `basic_stats` and `defensive_stats`. 

- It may be helpful to check out USING() in your JOIN instead of ON. Read about the differences [here](https://www.neilwithdata.com/join-using).

In [173]:
%%spark

#TODO: Create defensive_cleaned_sdf
defensive_stats_sdf.createOrReplaceTempView("defensive_stats")


query = '''SELECT *
           FROM defensive_stats
           '''
#TODO: Save the output sdf of spark.sql() as answer_defensive_sdf and convert to Pandas

defensive_sdf = spark.sql(query)
defensive_cleaned_sdf = defensive_sdf[defensive_sdf['Total Tackles']!='null']
defensive_cleaned_sdf = defensive_cleaned_sdf[defensive_cleaned_sdf['Games Played']!='null']
#defensive_cleaned_sdf 
#defensive_cleaned_sdf.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [174]:
%%spark

# 4 points
grader.grade(test_case_id = 'defensive_cleaned', answer = defensive_cleaned_sdf.toPandas())


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Correct! You earned 4/4 points. You are a star!

Your submission has been successfully recorded in the gradebook.

In [175]:
%%spark

#TODO: Create defensive_player_stats_sdf
defensive_cleaned_sdf.createOrReplaceTempView("defensive_cleaned")
basic_stats_sdf.createOrReplaceTempView("basic_stats")

query = '''SELECT *
           FROM defensive_cleaned as d
           inner join basic_stats as b using (`Player Id`)
           '''
#TODO: Save the output sdf of spark.sql() as answer_defensive_sdf and convert to Pandas

defensive_player_stats_sdf = spark.sql(query)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [176]:
%%spark

# 4 points
grader.grade(test_case_id = 'defensive_player_stats', answer = defensive_player_stats_sdf.toPandas())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Correct! You earned 4/4 points. You are a star!

Your submission has been successfully recorded in the gradebook.

## Step 2: Analysis!

### Step 2.1: The Robust Ratio

Now, we are interested in seeing the player with the highest total tackles / games played ratio from each team for each year and their corresponding experience. This involves derieving data from both the dataframes we created just now.

Your task is to first find the player with the highest ratio for each team per year from the defensive_player_stats_df, and then join that with the basic_stats_df to find the experience of that player.

**TODO**: Create an sdf called `best_tackles_sdf` that contains the above information. Then, make sure to only retain data pertaining to Team, player Id, Year, tackle_ratio, and experience. 

Remember we will sort the dataframe when grading so you can sort the elements however you wish (you don't need to if you don't want to).

You should have the following columns in your answer:`Experience, ID, Year, Ratio, Team`

In [177]:
%%spark

# TODO: Create best_tackles_sdf
defensive_player_stats_sdf.createOrReplaceTempView("defensive_player_stats")


query = '''WITH t1 as (SELECT `Experience`,`Player Id` as ID, `Year`, `Total Tackles`/`Games Played` as Ratio, `Team`
           FROM defensive_player_stats)
           Select Experience, ID,Year, Ratio, Team
           From t1

           '''
#TODO: Save the output sdf of spark.sql() as answer_defensive_sdf and convert to Pandas

best_tackles_sdf = spark.sql(query)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [178]:
%%spark
# Print out the first few rows to see if the dataframe looks reasonable
best_tackles_sdf.show(4)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------------------+----+------+----------------+
|Experience|                ID|Year| Ratio|            Team|
+----------+------------------+----+------+----------------+
| 5 Seasons|quinnjohnson/79593|2013|   0.0|Tennessee Titans|
| 5 Seasons|quinnjohnson/79593|2012|0.0625|Tennessee Titans|
| 5 Seasons|quinnjohnson/79593|2011|   0.0|Tennessee Titans|
| 5 Seasons|quinnjohnson/79593|2011|  null|  Denver Broncos|
+----------+------------------+----+------+----------------+
only showing top 4 rows

In [179]:
%%spark

# 10 points
grader.grade(test_case_id = 'tackle_ratio', answer = best_tackles_sdf.toPandas())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Correct! You earned 10/10 points. You are a star!

Your submission has been successfully recorded in the gradebook.


### Step 2.2: The Tremendous Tackles

**TODO**: Your task is to answer the following questions: 

1. Which team has the player which the best ratio? Save their ID to the variable `highest_tackle_ratio_person`.
2. Which team has the total highest ratio? Save this to the variable `highest_tackle_ratio_team`
3. For each player who has played for more than a year, what is the percentage change in their ratio from the first year they played to the last year they played? Save this df in `percentage_change_sdf`, and make sure to order by ID. This DF should just contain two columns- percentage change and ID, named as follows `Percentage Change, ID`: 


In [180]:
%%spark
best_tackles_sdf.createOrReplaceTempView("best_tackles")
#TODO: highest_tackle_ratio_person
query1 = ''' Select ID 
             From best_tackles 
             ORDER BY Ratio DESC
             LIMIT 1
           '''

highest_tackle_ratio_person= spark.sql(query1).toPandas()['ID'].to_list()[0]

#TODO: highest_tackle_ratio_team
query2 = ''' Select TEAM 
             From best_tackles 
             ORDER BY Ratio DESC
             LIMIT 1
           '''

highest_tackle_ratio_team= spark.sql(query2).toPandas()['TEAM'].to_list()[0]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [220]:
%%spark

# 2 points
grader.grade(test_case_id = 'best_player', answer = highest_tackle_ratio_person)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Correct! You earned 2/2 points. You are a star!

Your submission has been successfully recorded in the gradebook.
+--------------------+--------------+--------+----+-------------------+------------+-------------+------------+----------------+-----+-------+---------------+----+------------+---------+-------------+------------------+---+--------------------+---------+----------------+--------------+-------------------+----------+---------------+--------------+--------------------+--------------+------+--------+------------+------------+
|           Player Id|          Name|Position|Year|               Team|Games Played|Total Tackles|Solo Tackles|Assisted Tackles|Sacks|Safties|Passes Defended|Ints|Ints for TDs|Int Yards|Yards Per Int|Longest Int Return|Age|         Birth Place| Birthday|         College|Current Status|       Current Team|Experience|Height (inches)|   High School|High School Location|          Name|Number|Position|Weight (lbs)|Years Played|
+--------------------+----------

In [219]:
%%spark

# 2 points
grader.grade(test_case_id = 'best_team', answer = highest_tackle_ratio_team)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Correct! You earned 2/2 points. You are a star!

Your submission has been successfully recorded in the gradebook.

In [280]:
%%spark

#TODO Create % change table and save it as percentage_change_sdf
defensive_stats_sdf.createOrReplaceTempView("defensive_stats")

query1 = '''WITH t1 as(
            Select `Player Id` as ID, `Name`,`Year`, `Team`, `Games Played` as GP,`Total Tackles` as TT
            From defensive_stats
            )
            SELECT ID, Name, Year, Team, TT/GP as Ratio
            FROM t1

           '''
table1_sdf= spark.sql(query1)
table1_sdf.createOrReplaceTempView("table1")



query5 = '''WITH t1 as(
            Select `Player Id` as ID, `Name`,`Year`, `Team`, `Games Played` as GP,`Total Tackles` as TT
            From defensive_stats
            )
            SELECT ID, Name, Year, Team, TT/GP as Ratio
            FROM t1
            WHERE TT/GP != 0 or TT/GP IS NOT NULL 
           '''
table5_sdf= spark.sql(query5)
table5_sdf.createOrReplaceTempView("table5")

#last
query2 = '''
            SELECT ID as ID_l,Year as Year_l, Ratio as Ratio_l FROM table1 
            INNER JOIN(
            Select ID as ID_l, MAX(Year) as Year_l
            From table1
            GROUP BY ID_l)
            On ID = ID_l and Year = Year_l
            ORDER BY ID_l
           '''
table2_sdf= spark.sql(query2)

#first
query3 = '''WITH t2 as(
            SELECT ID as ID_F,Year as Year_F, Ratio as Ratio_F 
            FROM table5
            INNER JOIN(
            Select ID as ID_F, MIN(Year) as Year_F
            From table5
            GROUP BY ID_F)
            On ID = ID_F and Year = Year_F)
            SELECT *
            FROM t2
            ORDER BY ID_F
           '''
table3_sdf= spark.sql(query3)
table2_sdf.createOrReplaceTempView("table2")
table3_sdf.createOrReplaceTempView("table3")

query4 = '''With t1 as(
            SELECT ID_l as ID, 100*(Ratio_l-Ratio_F )/Ratio_F as  `Percentage Change`, Year_l-Year_F as Year_Diff
            FROM table2
            FULL OUTER JOIN(
            SELECT * FROM table3)
            On ID_l = ID_F)
            SELECT `ID`,`Percentage Change` 
            FROM t1
            WHERE Year_Diff>=1
            ORDER BY ID
           '''
percentage_change_sdf= spark.sql(query4)

table3_sdf.show()
table2_sdf.show()
percentage_change_sdf.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------+-------------------+
|                ID_F|Year_F|            Ratio_F|
+--------------------+------+-------------------+
|a'shawnrobinson/2...|  2016|              1.875|
|   a.j.bouye/2541162|  2013| 0.3333333333333333|
|     a.j.edds/496921|  2011| 1.3333333333333333|
|     a.j.edds/496921|  2011|                0.5|
|  a.j.feeley/2504566|  2001|                0.0|
| a.j.francis/2541707|  2015|                1.0|
| a.j.francis/2541707|  2015|                0.5|
|   a.j.green/2495450|  2011|0.13333333333333333|
|a.j.jefferson/494275|  2010|                0.5|
|   a.j.klein/2539982|  2013|             1.3125|
| a.j.schable/2506734|  2006| 0.9090909090909091|
| a.j.tarpley/2553605|  2015| 0.5714285714285714|
|aaronbeasley/2499587|  2001|               3.25|
|aaronburbridge/25...|  2016|             0.8125|
| aaroncolvin/2543501|  2014|                4.5|
| aarondobson/2539256|  2013|0.08333333333333333|
| aarondonald/2543485|  2014|                3.0|


In [281]:
%%spark

# Let's visualize the results
percentage_change_sdf.show(20)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+
|                  ID|  Percentage Change|
+--------------------+-------------------+
|   a.j.bouye/2541162| 1160.0000000000002|
|     a.j.edds/496921|             -100.0|
|     a.j.edds/496921| -99.99999999999999|
|     a.j.edds/496921|             -81.25|
|     a.j.edds/496921|              -50.0|
|  a.j.feeley/2504566|               null|
| a.j.francis/2541707|               null|
| a.j.francis/2541707|               null|
|   a.j.green/2495450|             -100.0|
|a.j.jefferson/494275|                0.0|
|   a.j.klein/2539982| 57.460317460317476|
|aaronbeasley/2499587| -51.64835164835165|
| aaroncolvin/2543501| -51.11111111111111|
| aarondobson/2539256|               null|
| aarondonald/2543485|-2.0833333333333335|
| aaronelling/2504975| -55.55555555555556|
|aaronfrancisco/25...|  672.6190476190476|
|  aaronglenn/2500813|-21.212121212121207|
|aaronkampman/2505138|             -100.0|
|  aaronlynch/2543650| 29.192546583850937|
+----------

In [282]:
%%spark

## AUTOGRADER Step 2.2: ##
percentage_change_sdf.createOrReplaceTempView("test_2_2")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [284]:
%%spark

# 12 points
grader.grade(test_case_id = 'percentage_change', answer = spark.sql("SELECT * FROM test_2_2 ORDER BY ID").toPandas())



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Correct! You earned 12/12 points. You are a star!

Your submission has been successfully recorded in the gradebook.

## 2.3 The Blessed Break

That last section was hard. And it's gonna get harder. Take a break. Sit back and relax for a minute. Listen to some music. 

**TODO**: In the cell below fill out the boolean variable `whatd_you_think` with `True` if you liked it or `False` if you didn't. You will be graded on your response.

In [92]:
whatd_you_think = 'True' # TODO


In [93]:
# 1 point
grader.grade(test_case_id = 'tunes', answer = whatd_you_think)

Correct! You earned 1/1 points. You are a star!

Your submission has been successfully recorded in the gradebook.


## Step 3: Formatting the Time Series Data


Our overarching goal is to create an annual time series model that will use the highest tackle ratio per year for each team to predict the highest tackle ratio for that team over the next season. The model is trained on a set of observations. Each observation contains the team name, and the highest tackle ratios for each year from 2001 to 2016. 

```
+----+-----+----------+---------+----------+---
|Team |2001 |...  |2008 |2009 |   ...   |2016 |
+----+-----+----------+---------+----------+---
|TEN  |...  |...  |...  |...  |   ...   |...  |
|SF   |...  |...  |...  |...  |   ...   |...  |
|...  |...  |...  |...  |...  |   ...   |...  |
+----+-----+----------+---------+----------+---
```



### Step 3.1: The Annual Arrangement

Your first task is to create the time series table, i.e. the `2001` through `2016` columns. This will involve reshaping `best_tackles_sdf`. Currently, `best_tackles_sdf` has columns `ID`, `Name`, `Team`, `Ratio` and `Year`. We want to group the rows together based on common `Team` and create new columns for the Ratio corresponding to each year.

**TODO**: Create an sdf called `raw_tackles_time_series_sdf` that has for a single team, the highest tackle ratio in 2001 through 2016. It is ok if for a given team you don't have a given year. However, ensure that for a given team, each year column has an entry, i.e. if no Ratio value is present for the year, the entry for that year column should be `0`. The format of the sdf is shown below: 
```
+----+-----+----------+---------+----------+----------+-------
|Team                |2001 |...  |2008 |2009 |   ...   |2016 |
+----+-----+----------+---------+----------+----------+-------
|Tennessee Titans    |...  |...  |...  |...  |   ...   |...  |
|San Francisco 49ers |...  |...  |...  |...  |   ...   |...  |
|...                 |...  |...  |...  |...  |   ...   |...  |
+----+-----+----------+---------+----------+----------+-------
```
_Hint_: This is a **fiddly and somewhat difficult** question. The tricky part is creating the additional columns of annual ratios, specifically when there are missing years. 

We suggest you look into `CASE` and `WHEN` statements in the [function list](https://spark.apache.org/docs/2.3.0/api/sql/index.html), and use these to **either** fill in a number for column (if appropriate) or put in a 0.

In [285]:
%%spark

# TODO: Create and save raw_tackles_time_series_sdf

# CASE() statements are SQL's equivalent of if else statements. WHEN a CASE is
# true THEN we define a function. ELSE we do another function and then END the
# statement.

# The query is a GROUP BY. We group data based on the same team, as in the
# previous step. We then do a CASE statement. This will seperate out the sets of
# data corresponding to the same year using the 'Year' volumn in the WHEN clause.
# If we have a piece of data, it will be the Ratio value for a given team in 
# a given year and we will save it with a corresponding column name. If there is
# no piece of data here, as per the question, we need to add a 0. This is the
# ELSE clause. Lastly, we do a MAX() to find the final Ratio.
best_tackles_sdf.createOrReplaceTempView("best_tackles")
query1 = '''
             WITH t2 AS(
             WITH t1 AS(Select Team, Year,MAX(Ratio) as Max_Ratio
             From best_tackles
             GROUP BY Team,Year)
             SELECT Team,
                 CASE WHEN Year == 2001 THEN Max_Ratio ELSE 0 end as `2001`,
                 CASE WHEN Year == 2002 THEN Max_Ratio ELSE 0 end as `2002`,
                 CASE WHEN Year == 2003 THEN Max_Ratio ELSE 0 end as `2003`,
                 CASE WHEN Year == 2004 THEN Max_Ratio ELSE 0 end as `2004`,
                 CASE WHEN Year == 2005 THEN Max_Ratio ELSE 0 end as `2005`,
                 CASE WHEN Year == 2006 THEN Max_Ratio ELSE 0 end as `2006`,
                 CASE WHEN Year == 2007 THEN Max_Ratio ELSE 0 end as `2007`,
                 CASE WHEN Year == 2008 THEN Max_Ratio ELSE 0 end as `2008`,
                 CASE WHEN Year == 2009 THEN Max_Ratio ELSE 0 end as `2009`,
                 CASE WHEN Year == 2010 THEN Max_Ratio ELSE 0 end as `2010`,
                 CASE WHEN Year == 2011 THEN Max_Ratio ELSE 0 end as `2011`,
                 CASE WHEN Year == 2012 THEN Max_Ratio ELSE 0 end as `2012`,
                 CASE WHEN Year == 2013 THEN Max_Ratio ELSE 0 end as `2013`,
                 CASE WHEN Year == 2014 THEN Max_Ratio ELSE 0 end as `2014`,
                 CASE WHEN Year == 2015 THEN Max_Ratio ELSE 0 end as `2015`,
                 CASE WHEN Year == 2016 THEN Max_Ratio ELSE 0 end as `2016`
             FROM t1)
             SELECT Team, MAX(`2001`) as `2001`,MAX(`2002`) as `2002`,MAX(`2003`) as `2003`,MAX(`2004`) as `2004`,MAX(`2005`) as `2005`
             ,MAX(`2006`) as `2006`,MAX(`2007`) as `2007`,MAX(`2008`) as `2008`,MAX(`2009`) as `2009`,MAX(`2010`) as `2010`,
             MAX(`2011`) as `2011`,MAX(`2012`) as `2012`,MAX(`2013`) as `2013`,MAX(`2014`) as `2014`,MAX(`2015`) as `2015`,MAX(`2016`) as `2016`
             FROM t2 
             GROUP BY Team
             
           '''
raw_tackles_time_series_sdf= spark.sql(query1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [207]:
%%spark
raw_tackles_time_series_sdf.createOrReplaceTempView("test_3_1")
test_3_1_sdf = spark.sql("SELECT * FROM test_3_1 ORDER BY Team DESC LIMIT 10")
test_3_1_sdf.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+
|                Team|             2001|             2002|              2003|             2004|             2005|             2006|             2007|             2008|              2009|             2010|             2011|             2012|             2013|              2014|             2015|             2016|
+--------------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+
| Washington Redskins|7.071428571428571|           6.1875|

In [286]:
%%spark

# 14 points
grader.grade(test_case_id = 'arrangement', answer = test_3_1_sdf.toPandas())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Correct! You earned 14/14 points. You are a star!

Your submission has been successfully recorded in the gradebook.

### Step 3.2 The Clairvoyant Cleaning

We now want to format the `Team` names to an appropriate abbreviation. The formatting will consist of 2 steps.

#### Step 3.2.1 The Ubiquitous UDF

The teams in our dataset are defined by their full names. We think that it would be *cleaner* to have names represented by using abbreviations. Often times when using Spark, there may not be a built-in SQL function that can do the operation we desired. Instead, we can create one on our own with a user-defined function (udf).

A udf is defined as a normal Python function and then registered to be used as a Spark SQL function. Your task is to create a udf, `NAME_TO_ABBV()` that will convert the `Team` field in `raw_tackles_time_series_sdf` to their appropriate abbreviations. This will be done using the provided `name_to_abbv_dict` dictionary. We are only interested in the teams in that dictionary.

**TODO**: Fill out the function `name_to_abbv()` below. Then use `spark.udf.register()` to register it as a SQL function. The command is provided. ***You do not need to edit it***. Note, we have defined the udf as returning `StringType()`. Ensure that your function returns this. You must also deal with any potential `null` cases.

In [287]:
%%spark

# Dictionary linking stock ticker symbols to their names
name_to_abbv_dict = {'Tennessee Titans': 'TEN',
                     'Denver Broncos': 'DEN',
                     'Green Bay Packers': 'GBP',
                     'Pittsburgh Steelers': 'PIT',
                     'Chicago Bears': 'CHI',
                     'Philadelphia Eagles': 'PHI',
                     'Indianapolis Colts': 'IND',
                     'Arizona Cardinals': 'ARI',
                     'Seattle Seahawks': 'SEA',
                     'Baltimore Ravens': 'BAR',
                     'Carolina Panthers': 'CAR',
                     'Kansas City Chiefs': 'KAN',
                     'Oakland Raiders': 'OAK',
                     'St. Louis Rams': 'SLR',
                     'Atlanta Falcons': 'ATL',
                     'New Orleans Saints': 'NOS',
                     'San Francisco 49ers': 'SFF',
                     'New England Patriots': 'NEP',
                     'Buffalo Bills': 'BUF',
                     'Los Angeles Rams': 'LAR',
                     'Dallas Cowboys': 'DAL',
                     'Minnesota Vikings': 'MIN',
                     'Detroit Lions': 'DET',
                     'Washington Redskins': 'WAS',
                     'Jacksonville Jaguars': 'JAC',
                     'New York Giants': 'NYG',
                     'Tampa Bay Buccaneers': 'TBB',
                     'Cleveland Browns': 'CLE',
                     'Houston Texans': 'HOT',
                     'Los Angeles Raiders': 'LOS',
                     'Miami Dolphins': 'MIA',
                     'Houston Oilers': 'HOU',
                     'Cincinnati Bengals': 'CIN',
                     'San Diego Chargers': 'SDC',
                     'Tennessee Oilers': 'TEO',
                     'New York Jets': 'NYJ',
                     'Phoenix Cardinals': 'PHO',
                     'St. Louis Cardinals': 'SLC',
                     'Baltimore Colts': 'BAC'}

# TODO: Fill out name_to_abbv() and register it as a udf.

# In UDFs we have to cover all possible output cases, or else the function will
# crash. Specifically, this means we need to handle the case when "name" is
# not in "name_to_abbv_dict". We use a try and except statement to return null
# for this case.

def name_to_abbv(name):
    try: 
        a = name_to_abbv_dict[name]
        return a 
    except:
        return None


# Register udf as a SQL function. DO NOT EDIT
spark.udf.register("NAME_TO_ABBV", name_to_abbv, StringType())



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<function name_to_abbv at 0x7ff18bf7b9e0>

Submit a tuple to the autograder for the ticker value of San Franscisco 49ers and Washington Commanders. If the name value in the table, set it to a string equal to "None"

In [288]:
%%spark

## AUTOGRADER Step 3.2.1: ##

to_submit = ((str(name_to_abbv("San Francisco 49ers")),str(name_to_abbv("Washington Commanders"))))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [289]:
%%spark

# 2 points
grader.grade(test_case_id = 'clairvoyant', answer = to_submit)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Correct! You earned 2/2 points. You are a star!

Your submission has been successfully recorded in the gradebook.

#### Step 3.2.2: The Clean Conversion

With our new `NAME_TO_ABBV()` function we will begin to wrangle `raw_tackles_time_series_sdf`.

**TODO**: Create an sdf called `tackles_time_series_sdf` as follows. Convert all the team names in `raw_tackles_time_series_sdf` to the appropriate abbreviations and save it as `Team`. Drop any team abbreviations that do not appear in `name_to_abbv_dict`. Using .dropna() is acceptable instead of IS NOT NULL. The final df should be in the format shown below:

```
+----+-----+----------+---------+----------+----------+--
|Team |2000 |2001 |   ...   |2008 |2009 |   ...   |2016 |
+----+-----+----------+---------+----------+----------+--
|TEN  |...  |...  |   ...   |...  |...  |   ...   |...  |
|SFF  |...  |...  |   ...   |...  |...  |   ...   |...  |
|...  |...  |...  |   ...   |...  |...  |   ...   |...  |
+----+-----+----------+---------+----------+----------+--
```

In [290]:
%%spark

# Format the "Team" column using our UDF, NAME_TO_ABBV.
# TODO: Create and save tackles_time_series_sdf


best_tackles_sdf.createOrReplaceTempView("best_tackles")
query2 = '''
             WITH t2 AS(
             WITH t1 AS(Select Team, Year,MAX(Ratio) as Max_Ratio
             From best_tackles
             GROUP BY Team,Year)
             SELECT Team,
                 CASE WHEN Year == 2001 THEN Max_Ratio ELSE 0 end as `2001`,
                 CASE WHEN Year == 2002 THEN Max_Ratio ELSE 0 end as `2002`,
                 CASE WHEN Year == 2003 THEN Max_Ratio ELSE 0 end as `2003`,
                 CASE WHEN Year == 2004 THEN Max_Ratio ELSE 0 end as `2004`,
                 CASE WHEN Year == 2005 THEN Max_Ratio ELSE 0 end as `2005`,
                 CASE WHEN Year == 2006 THEN Max_Ratio ELSE 0 end as `2006`,
                 CASE WHEN Year == 2007 THEN Max_Ratio ELSE 0 end as `2007`,
                 CASE WHEN Year == 2008 THEN Max_Ratio ELSE 0 end as `2008`,
                 CASE WHEN Year == 2009 THEN Max_Ratio ELSE 0 end as `2009`,
                 CASE WHEN Year == 2010 THEN Max_Ratio ELSE 0 end as `2010`,
                 CASE WHEN Year == 2011 THEN Max_Ratio ELSE 0 end as `2011`,
                 CASE WHEN Year == 2012 THEN Max_Ratio ELSE 0 end as `2012`,
                 CASE WHEN Year == 2013 THEN Max_Ratio ELSE 0 end as `2013`,
                 CASE WHEN Year == 2014 THEN Max_Ratio ELSE 0 end as `2014`,
                 CASE WHEN Year == 2015 THEN Max_Ratio ELSE 0 end as `2015`,
                 CASE WHEN Year == 2016 THEN Max_Ratio ELSE 0 end as `2016`
             FROM t1)
             SELECT name_to_abbv(Team) as Team, MAX(`2001`) as `2001`,MAX(`2002`) as `2002`,MAX(`2003`) as `2003`,MAX(`2004`) as `2004`,MAX(`2005`) as `2005`
             ,MAX(`2006`) as `2006`,MAX(`2007`) as `2007`,MAX(`2008`) as `2008`,MAX(`2009`) as `2009`,MAX(`2010`) as `2010`,
             MAX(`2011`) as `2011`,MAX(`2012`) as `2012`,MAX(`2013`) as `2013`,MAX(`2014`) as `2014`,MAX(`2015`) as `2015`,MAX(`2016`) as `2016`
             FROM t2 
             GROUP BY Team
             
           '''
tackles_time_series_sdf= spark.sql(query2)




FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [291]:
%%spark

## AUTOGRADER Step 3.2.2: ##

tackles_time_series_sdf.createOrReplaceTempView("test_3_2_2")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [292]:
%%spark

# 10 points
grader.grade(test_case_id = 'conversion', answer = spark.sql("SELECT * FROM test_3_2_2 ORDER BY Team LIMIT 10").toPandas())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Correct! You earned 10/10 points. You are a star!

Your submission has been successfully recorded in the gradebook.

And there you go! Now we have formatted a simple time series dataset for the highest tackle ratios of all teams.

## Step 4: The Super... Bacon?

Let's introduce a fun little concept called the Bacon Number! The Bacon number of an actor or actress is the number of degrees of separation they have from actor Kevin Bacon, as defined by the game known as Six Degrees of Kevin Bacon. For example, Kevin Bacon's Bacon number is 0. If an actor works in a movie with Kevin Bacon, the actor's Bacon number is 1. If an actor works with an actor who worked with Kevin Bacon in a movie, the first actor's Bacon number is 2, and so forth.

How do we implement this for NFL though? Let's use a dataset specifically based on the history of the Super Bowl, and find a number we're calling "The Super Bacon." We define this number as follows: if team A has beaten team B, and team B has beaten team C, then the super bacon of C with respect to A will be 2.

Now to calculate this number, we'll use the concepts of graphs and BFS!


*For this section, we will be using Pyspark instead of Spark on the EMR cluster. The syntax remains the same, except you don't need to use the `%%spark` call at the top of each code cell.
Run the setup cells for Pyspark below.*

In [293]:
from pyspark.sql import SparkSession

appName = "PySpark"

# Create Spark session
spark = SparkSession.builder.appName(appName).getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/25 21:23:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### 4.1 “Traversing” a Graph

Before we jump to finding the Super Bacons for the teams, let's review how BFS works.

In [294]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark import SparkConf
from pyspark import SparkContext


#### 4.1.1 Intro to Distributed Breadth-First Search


To start off, we will be implementing a graph traversal algorithm known as Breadth First Search. It works in a way that's equivalent to how a stain spreads on a white t-shirt. Take a look at the graph below:

<p align = "center">
<img src = "https://imgur.com/WU3AUwg.png" width= "600" align ="center"/>

* Consider starting BFS from point A (green). This is considered the starting frontier/singular origin node.
* The first round of BFS would involve finding all the nodes directly reachable from A, namely B-F (blue circles). These blue nodes make up the next frontier at depth 1 away from our starting node A.
* The second round would then be identifying the red nodes which are the neighbors of the blue nodes. Now, the red nodes all belong to a frontier 2 depth away from A. Note that node A is also a neighbor of a blue node. However, since it has already been visited, it does not get added to this frontier.

This process continues until all the nodes in the graph have been visited.
If you would like to learn more about BFS, we highly suggest looking [here](https://www.tutorialspoint.com/data_structures_algorithms/breadth_first_traversal.html).


We will now be implementing **spark_bfs(G, N, d)**, our spark flavor of BFS that takes a graph **G**, a set of origin nodes **N**, and a max depth **d**.

In order to write a successful BFS function, you are going to need to figure out 
1. how to keep track of nodes that we have visited
2. how to properly find all the nodes at the next depth
3. how to avoid cycles and ensure that we do not constantly loop through the same edges (take a look at J-K in the graph)


#### BFS vs. DFS Animation [ADDITIONAL INFO] 
Here's a helpful visualzation to understand the difference between depth and breadth first search! (Source: Wikimedia Commons)

#### **BFS**
![BFS](https://upload.wikimedia.org/wikipedia/commons/5/5d/Breadth-First-Search-Algorithm.gif)
#### **DFS**
![DFS](https://upload.wikimedia.org/wikipedia/commons/7/7f/Depth-First-Search.gif)

#### 4.1.2 Implement One Traversal

To break down this process, let's think about how we would implement a single traversal of the graph. That is given the green node in the graph above, how are we going to get the blue nodes?


Consider the simple graph below **which is different from the graph in the image above**:

In [295]:

simple = [('A', 'B'),
         ('A', 'C'),
         ('A', 'D'),
         ('C', 'F'),
         ('F', 'A'),
         ('B', 'G'),
         ('G', 'H'),
         ('D', 'E')]

simple_dict = {'from_node': ['A', 'A', 'A', 'C', 'F', 'B', 'G', 'D'],
       'to_node': ['B', 'C', 'D', 'F', 'A', 'G', 'H', 'E']}

simple_graph_df = pd.DataFrame.from_dict(simple_dict)
simple_graph_sdf = spark.createDataFrame(simple_graph_df)
simple_graph_sdf.show()

+---------+-------+
|from_node|to_node|
+---------+-------+
|        A|      B|
|        A|      C|
|        A|      D|
|        C|      F|
|        F|      A|
|        B|      G|
|        G|      H|
|        D|      E|
+---------+-------+



As you can see, each row of this dataframe represents an edge between two nodes Here, we are looking at a **directed graph**, which means that A-->B  does not represent the same edge as B-->A.

Let's define our starting node as follows:

In [296]:
smallOrig = [{'node': 'A'}]

Then, bfs with graph G, starting from smallOrig to depth 1, or  **spark_bfs(G, smallOrig, 1)** would output as follows:

In [297]:
simple_1_round_dict = {'node': ['B', 'D', 'C', 'A'],
       'distance': [1, 1, 1, 0]}
simple_1_round_bfs_df = pd.DataFrame.from_dict(simple_1_round_dict)
simple_1_round_bfs_sdf = spark.createDataFrame(simple_1_round_bfs_df)
simple_1_round_bfs_sdf.show()


+----+--------+
|node|distance|
+----+--------+
|   B|       1|
|   D|       1|
|   C|       1|
|   A|       0|
+----+--------+



As you can see, this dataframe logs each node with its corresponding distance away from A. Moreover, we also know that these nodes are **visited**. 

Hopefully, you can see how we can use our original graph and this new information to find the nodes at depth two. 

This is exactly what we will try to accomplish with **spark_bfs_1_round(visited_nodes)** which will ultimately be the inner function of **spark_bfs** that we use to perform exactly one traversal of a graph.

**TODO**: Write **spark_bfs_1_round(visted_nodes)** that takes the currently dataframe of visited_nodes, performs one round of BFS, and returns an updated visited nodes dataframe. You should assume that a temporary sdf G already exists.

In [306]:
def spark_bfs_1_round(visited_nodes):
  """
  :param visited_nodes: dataframe with columns node and distance
  :return: dataframe of updated visuted nodes, with columns node and distance
  """
  # TODO: Complete this function to implement 1 round of BFS
  next_sdf =  visited_nodes
  visited_sdf = visited_nodes
  #second round
  edge_sdf = next_sdf.join(simple_graph_sdf, (next_sdf['node'] == simple_graph_sdf['from_node']),'inner').cache()
  next_sdf = edge_sdf.select(edge_sdf['to_node'].alias('node'))
  from_node = edge_sdf.select(edge_sdf['from_node'].alias('node'))
  next_sdf = next_sdf.union(from_node).drop_duplicates(['node']).cache()
  next_sdf = next_sdf.join(visited_sdf, visited_sdf['node'] == next_sdf['node'], 'leftanti')
  visited_sdf = visited_sdf.union(next_sdf.withColumn('distance',F.lit(2))).drop_duplicates().cache()
  return visited_sdf

Now, run the inner function on **simple_1_round_bfs_sdf** (i.e. result of 1 round of BFS on the simple graph) and store the results in **simple_bfs_result**. This is ultimately what the output of BFS to depth 2 should look like.

In [307]:
# TODO: Run spark_bfs_1_round on simple_1_round_bfs_sdf
simple_bfs_test_sdf = spark_bfs_1_round(simple_1_round_bfs_sdf)

#simple_1_round_bfs_sdf.show()

22/10/25 21:28:58 WARN CacheManager: Asked to cache already cached data.
22/10/25 21:28:58 WARN CacheManager: Asked to cache already cached data.


Convert this result to Pandas, sorted by the node (ascending), and submit it to the autograder.

**HINT:** Make sure distance is formatted as a number!

In [308]:
# TODO: Convert simple_bfs_result to Pandas sorted by node
simple_bfs_test = simple_bfs_test_sdf.toPandas().sort_values(by = 'node',ascending = True)
simple_bfs_test

,node,distance
6,A,0
2,B,1
3,C,1
5,D,1
1,E,2
4,F,2
0,G,2


In [309]:
# 6 points
grader.grade(test_case_id = 'checksimpleBFS', answer = simple_bfs_test)

Correct! You earned 6/6 points. You are a star!

Your submission has been successfully recorded in the gradebook.


#### 4.1.3 Full BFS Implementation

Now, we will fully implement **spark_bfs**. This function should iteratively call your implemented version of **spark_bfs_1_round** and ultimately return the output of this function at **max_depth**.

You are also responsible for initializing the starting dataframe, that is converting the list of origin nodes into a spark dataframe with the nodes logged at distance 0.

Consider the following: 

```python
schema = StructType([
            StructField("node", StringType(), True)
        ])
my_sdf = spark.createDataFrame(origins, schema)
```

The schema ultimately specifies the structure of the Spark DataFrame with a string `node` column. It then calls **spark.createDataFrame** to map this schema to the **origins** nodes. Also, you are responsible for ensuring that a view of your graph is available within this function. (Note: you will also need to add in a distance column)

**TODO:** implement **spark_bfs(G,origins,max_depth)** and run on **teams_graph_sdf** initalized in 4.3. Note: you may want to run tests on the **simple_graph** example as the `teams_graph_sdf` will take quite some time to run.

These imports might be useful: 
`from pyspark.sql.types import StructType, StructField, StringType, IntegerType`

In [319]:
# TODO: iterative search over directed graph
# Worth 5 points directly, but will be needed later
def spark_bfs(G, origins, max_depth):
  """ runs distributed BFS to a specified max depth

  :param G: graph dataframe from 4.3
  :param origins: list of origin nodes stored as {"node": nodeValue}
  :param max_depth: integer value of max depth to run BFS to
  :return: dataframe with columns node, distance of all visited nodes
  """
  next_sdf = spark.createDataFrame(origins, schema)
  visited_sdf = spark.createDataFrame(origins, schema).withColumn('distance',F.lit(0))
  for distance in range(max_depth):
      edge_sdf = next_sdf.join(G, (next_sdf['node'] == G['from_node']),'inner').cache()
      next_sdf = edge_sdf.select(edge_sdf['to_node'].alias('node'))
      from_node = edge_sdf.select(edge_sdf['from_node'].alias('node'))
      next_sdf = next_sdf.union(from_node).drop_duplicates(['node']).cache()
      next_sdf = next_sdf.join(visited_sdf, visited_sdf['node'] == next_sdf['node'], 'leftanti')
      visited_sdf = visited_sdf.union(next_sdf.withColumn('distance',F.lit(distance+1))).drop_duplicates().cache()
  return visited_sdf

In [320]:
schema = StructType([StructField("node",StringType(),True)])

Test that this function works on the simple example. 

In [321]:
simple_bfs_iterative_result = spark_bfs(simple_graph_sdf, smallOrig, 3)
simple_bfs_iterative_result.show()

+----+--------+
|node|distance|
+----+--------+
|   G|       2|
|   F|       2|
|   E|       2|
|   A|       0|
|   C|       1|
|   B|       1|
|   D|       1|
|   H|       3|
+----+--------+



### Step 4.2: History of the Super Bowl

Time to look at the Super Bowl history dataset. Here we have data for all Super Bowls from 1967 to 2020. Let's load the data to **superbowl_sdf** and see how it looks.

In [322]:
from pyspark import SparkFiles
spark.sparkContext.addFile("https://storage.googleapis.com/penn-cis5450/superbowl.csv")
superbowl_sdf = spark.read.csv(SparkFiles.get("superbowl.csv"), header=True, inferSchema=True)

In [323]:
superbowl_sdf.show()

+-----------+------------+--------------------+----------+--------------------+---------+---------------+--------------------+---------------+----------+
|       Date|          SB|              Winner|Winner Pts|               Loser|Loser Pts|            MVP|             Stadium|           City|     State|
+-----------+------------+--------------------+----------+--------------------+---------+---------------+--------------------+---------------+----------+
| Feb 2 2020|    LIV (54)|  Kansas City Chiefs|        31| San Francisco 49ers|       20|Patrick Mahomes|   Hard Rock Stadium|  Miami Gardens|   Florida|
| Feb 3 2019|   LIII (53)|New England Patriots|        13|    Los Angeles Rams|        3| Julian Edelman|Mercedes-Benz Sta...|        Atlanta|   Georgia|
| Feb 4 2018|    LII (52)| Philadelphia Eagles|        41|New England Patriots|       33|     Nick Foles|   U.S. Bank Stadium|    Minneapolis| Minnesota|
| Feb 5 2017|     LI (51)|New England Patriots|        34|     Atlanta Falco

Do you know which teams have been the most successful during this period? Let's find out!

**TODO:** Calculate the number of times each team appears in the *Winner* column, and store this **count_sdf**. This should have two columns: *Winner* and *win_count*.

In [324]:
# TODO
count_df =superbowl_sdf.cube("Winner").count()


Convert the dataframe to pandas, naming it `count_df` and pass it to the grader.

In [325]:
# TODO: create count_df 
count_df = count_df.toPandas()

In [326]:
# 3 points
grader.grade(test_case_id = 'checkSuperBowlCount', answer = count_df)

Correct! You earned 3/3 points. You are a star!

Your submission has been successfully recorded in the gradebook.


Now that we have the Super Bowl dataset, let's convert it to a graph sdf just like the one we had in section 4.1 (P.S. it's not as hard as it sounds).

**TODO:** Create **teams_graph_sdf** that has the columns *from_node* and *to_node*. from_node has all the entries from the *Winner* column and to_node has all the entries from the *Loser* column.

In [327]:
# TODO:
 #spark.createDataFrame(simple_graph_df)
superbowl_sdf.createOrReplaceTempView("superbowl")
query1 = ''' Select Winner as from_node, Loser as to_node
             From superbowl
           '''
    
teams_graph_sdf = spark.sql(query1)

In [328]:
# 2 points
grader.grade(test_case_id = 'checkSuperBowlGraph', answer = teams_graph_sdf.toPandas())

Correct! You earned 2/2 points. You are a star!

Your submission has been successfully recorded in the gradebook.


We have our graph ready! All that's left is running full BFS on it find the Super Bacons.

### Step 4.3: The Super Bacon Search

In the previous section, we found out that the New England Patriots and the Pittsburgh Steelers have been the most successful teams in the 54 years. So let's find out the Super Bacons of teams with respect to the New England Patriots.

First, we'll create an origin node using the New England Patriots.

In [329]:
orig = [{'node': 'New England Patriots'}]

Now, run the **spark_bfs()** function on **teams_graph_sdf** using this origin node and a max depth of 5. Store the result in **bfs_5**.

In [330]:
# TODO
answer_df = spark_bfs(teams_graph_sdf, orig, 5)

In [331]:
bfs_5 = answer_df.sort(F.col('node'))
bfs_5.show()

+--------------------+--------+
|                node|distance|
+--------------------+--------+
|   Arizona Cardinals|       5|
|     Atlanta Falcons|       1|
|   Carolina Panthers|       1|
|      Dallas Cowboys|       5|
|      Denver Broncos|       2|
|   Green Bay Packers|       3|
|  Kansas City Chiefs|       4|
|    Los Angeles Rams|       1|
|   Minnesota Vikings|       5|
|New England Patriots|       0|
|     Oakland Raiders|       4|
| Philadelphia Eagles|       1|
| Pittsburgh Steelers|       4|
| San Francisco 49ers|       5|
|    Seattle Seahawks|       1|
|      St. Louis Rams|       1|
|    Tennessee Titans|       2|
+--------------------+--------+



In [332]:
# 15 points
grader.grade(test_case_id = 'checkBFS', answer = bfs_5.toPandas())

Correct! You earned 15/15 points. You are a star!

Your submission has been successfully recorded in the gradebook.


Voila! We're all done! One last thing, as we predicted before, you're a star.

In [333]:
# Run this cell to convert homework3.ipynb to homework3.py (see submission instructions below)

!jupyter nbconvert --to script 'homework3.ipynb' 

[NbConvertApp] WARNING | Config option `kernel_spec_manager_class` not recognized by `NbConvertApp`.
[NbConvertApp] WARNING | pattern 'homework3.ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to 

# HW Submission

Before you submit on Gradescope (you must submit your notebook to receive credit):


1.   Restart and Run-All to make sure there's nothing wrong with your notebook
2.   **Double check that you have the correct PennID (all numbers) in the autograder**. 
3. Make sure you've run all the PennGrader cells
4. Make sure that this notebook is named `homework3.ipynb` and then run the cell above to export it to a python file (`homework3.py`). If needed, **rename** the files to make sure they are **named exactly** "homework3.ipynb" and "homework3.py" respectively and upload them to Gradescope 

**Let the course staff know ASAP if you have any issues submitting, but otherwise best of luck!**